# Cycle benchmarking

- With parallel CNOT

- Apply 9 different measurement settings



In [1]:
import numpy as np
import sys, json, copy, time, pickle, random
from concurrent.futures import ThreadPoolExecutor
import qiskit
from qiskit import IBMQ, QuantumCircuit, execute
from qiskit_aer import StatevectorSimulator, AerSimulator
from qiskit_aer.noise import NoiseModel, pauli_error, amplitude_damping_error, ReadoutError
# from qiskit.providers.ibmq.managed import IBMQJobManager, ManagedJobSet
# from qiskit.providers.ibmq.apiconstants import ApiJobShareLevel
from qiskit.quantum_info.operators.symplectic import pauli
sys.path.append(r"/Users/yzhu/yzhu_work/gates projects/EAB/EAB_code from Senrui")
from qubit_map import my_qubit_maps
import matplotlib.pyplot as plt
from scipy.stats import sem, unitary_group
from scipy import sparse
import CB_submit, CB_process
from statistics import stdev
import itertools
from qiskit.compiler import transpile
from qiskit_ionq import IonQProvider
from qiskit.quantum_info import Pauli

## Additional functions

In [2]:
## Additional functions

def int_to_pauli(i,n):
    p = np.base_repr(i,base=4)
    p = '0'*(n-len(p)) + p
    p = p.replace('0','I').replace('1', 'X').replace('2', 'Y').replace('3', 'Z')
    return p

def commute(p,q):
    c = 1
    n = len(p)
    for i in range(n):
        if p[i] != 'I' and q[i] != 'I':
            if p[i] != q[i]:
                c *= -1
    return c

def fidelity_to_error(pauli_fidelity,n):
    N = 4**n
    pauli_error = {}
    for i in range(N):
        p = int_to_pauli(i,n)
        pauli_error[p] = 0
        for j in range(N):
            q = int_to_pauli(j,n)
            pauli_error[p] += pauli_fidelity[q] * commute(p,q) / N
    return pauli_error
    

## Data generation

In [3]:
import os
os.environ["IONQ_API_KEY"] = "2IrrtuzJntPCcKvJzkKczpA1TJKTH9LO"
my_api_key = os.getenv("IONQ_API_KEY")
# print (my_api_key)
provider = IonQProvider(my_api_key)

backend = provider.get_backend("ionq_simulator")


In [4]:

use_density_matrix = False # density matrix based / measurement based simulation

# choose one
use_ibmq = False
use_stabilizer_simulator = True # whether stabilizer simulator is used (valid only for Pauli noise)
use_density_matrix_sample = False # use density matrix simulation, but returns samples
use_state_vector_sample = False # use state vector simulation, which returns samples

# parameters: n, Lrange, C, shots
use_readout_error = False
shots = 2000
n = 2 # num of qubit
n_total = n
# Lrange = list(range(2,39,4)) # len = 10
# Lrange = [2**x for x in range(1,6)]
Lrange=[2,8]
C = 1
batch = 1 # not used
gset = "Pauli"
q = my_qubit_maps['local']
repeat = [1 for k in Lrange] # not used
periodic = False # not used

# For CB, need to decide measurement basis
pauli_sample_list = [''.join(s) for s in itertools.product(['X','Y','Z'], repeat = n)]
# pauli_sample_list = ['XZ','ZX']


# For specific IBM device structure
# TODO: put these in qubit_map.py


clifford_layer = 'XX'
# clifford_layer = 'Id'

# Simulation parameters
eps = 0 # two parallel Cnot ~ 2%
# eps_amp = 0 # SPAM error at CNOT
eps_readout = 0.01 # readout bitflip ~ 1%
# eps_cross = 0.025 # SPAM error at CNOT

noise_model = NoiseModel()
# # noise_model.add_all_qubit_quantum_error(pauli_error([('I',1-eps),('X',eps)]),['id', 'x', 'y','z'])
noise_model.add_all_qubit_quantum_error(pauli_error([('II',(1-eps)**2),('XX',eps**2),('IX',eps*(1-eps)),('XI',eps*(1-eps))]),['cx'])

# # phase/amplitude damping error
# amp_noise_1q = amplitude_damping_error(eps_amp)
# noise_model.add_all_qubit_quantum_error(amp_noise_1q.tensor(amp_noise_1q),['cx'])

# # crosstalk between CNOTs (choose it to be ZZ)
# crosstalk_noise = pauli_error([('II',1-eps_cross),('ZZ',eps_cross)])
# for i in range(n-2):
#     noise_model.add_nonlocal_quantum_error(crosstalk_noise, ['cx'], [i,i+1], [i+1,i+2])
# noise_model.add_nonlocal_quantum_error(crosstalk_noise, ['cx'], [n-2,n-1], [n-1,0])
# noise_model.add_nonlocal_quantum_error(crosstalk_noise, ['cx'], [n-1,0], [0,1])

# readout error
if use_readout_error is True:
    p0given1 = eps_readout
    p1given0 = eps_readout
    readout_noise_1q = ReadoutError([[1 - p1given0, p1given0], [p0given1, 1 - p0given1]])
    noise_model.add_all_qubit_readout_error(readout_noise_1q)



if use_ibmq is False:
    filename = 'sim_cb_cnot_2022oct'
else:
    filename = 'exp_cb_cnot_2022oct'
    job_manager = IBMQJobManager()


# backend = AerSimulator(method='stabilizer')
# if use_density_matrix or use_density_matrix_sample:
#     backend = AerSimulator(method='density_matrix', noise_model=noise_model)
# elif use_state_vector_sample:
#     backend = AerSimulator(method='statevector', noise_model=noise_model)
# elif use_stabilizer_simulator:
#     # backend = AerSimulator(method='stabilizer', noise_model=noise_model)
#     backend = AerSimulator(method='stabilizer')
# elif use_ibmq:
#     provider = IBMQ.load_account()
#     backend = provider.get_backend('ibmq_manila')
# else:
#     assert 1==0

# print("CB, ", "n=%d" % n)
# print("simulation method:", backend.configuration().description)
if use_density_matrix:
    print("Density matrix based simulation")
else:
    print("Measurement based simulation, shots = %d" % shots)

if use_readout_error is True:
    print("Measurement bitflip rate = %f" % eps_readout)
else:
    print("No readout error")


data = {}
token = ''.join(random.choice([str(j) for j in range(10)]) for i in range(10))
filename += '_' + token
data["token"] = token
data["n"] = n
data["pauli_sample_list"] = pauli_sample_list



for pauli_sample in pauli_sample_list:
    print("CB for ",pauli_sample)
    # generate CB circuit:
    cb_data, cb_circ_all = CB_submit.submit_cb_XX(n,n_total,clifford_layer = clifford_layer,Lrange=Lrange,C=C,batch=batch, qubit_map=q, pauliList=pauli_sample , gset=gset,repeat=repeat,periodic=periodic,use_density_matrix=use_density_matrix)
    print("created %d circuits" % len(cb_circ_all[0]))

    print(cb_circ_all[0][0]) # print a typical example

    # print(transpile(cb_circ_all[0][0],basis_gates=backend.configuration().basis_gates))
    # print(transpile(cb_circ_all[0][0],optimization_level=3,basis_gates=backend.configuration().basis_gates,coupling_map=backend.configuration().coupling_map,initial_layout=q))

    # sys.exit(0)

    if use_density_matrix is True:
        job = backend.run(cb_circ_all[0], shots=1, max_parallel_experiments=0) 
    elif use_ibmq is False:
        job = backend.run(cb_circ_all[0], shots=shots, max_parallel_experiments=0, memory = True, noise_model="harmony") 
    else:
        job = backend.run(transpile(cb_circ_all[0],basis_gates=backend.configuration().basis_gates), shots=shots, memory = True,noise_model="harmony") 
    result = job.result()
    cb_data["result"] = [result]

    cb_data["parameters"] = {}
    cb_data["parameters"]['n'] = n 
    cb_data["parameters"]['n_total'] = n_total
    cb_data["parameters"]['shots'] = shots 
    cb_data["parameters"]['Lrange'] = Lrange 
    cb_data["parameters"]['C'] = C
    cb_data["parameters"]['eps_readout'] = eps_readout
    cb_data["parameters"]['repeat'] = repeat

    data[pauli_sample] = cb_data

# if use_ibmq is False:
#     cb_data["result"] = [result]
# else:
#     cb_data["job_set_id"] = job_set_id

# test: data saving
# print(cb_data)


# with open('data/' + filename, 'wb') as outfile:
#     pickle.dump(data, outfile)

# print(token)

Measurement based simulation, shots = 2000
No readout error
CB for  XX
created 2 circuits
     ┌───┐ ░ ┌───┐     ┌───┐┌───┐┌───┐┌───┐     ┌───┐┌───┐┌───┐ ░ ┌───┐     »
q_0: ┤ H ├─░─┤ X ├──■──┤ Z ├┤ H ├┤ X ├┤ Z ├──■──┤ S ├┤ H ├┤ X ├─░─┤ X ├──■──»
     ├───┤ ░ ├───┤┌─┴─┐└───┘└───┘└───┘└───┘┌─┴─┐├───┤├───┤├───┤ ░ ├───┤┌─┴─┐»
q_1: ┤ H ├─░─┤ X ├┤ X ├────────────────────┤ X ├┤ H ├┤ S ├┤ H ├─░─┤ X ├┤ X ├»
     └───┘ ░ └───┘└───┘                    └───┘└───┘└───┘└───┘ ░ └───┘└───┘»
c: 2/═══════════════════════════════════════════════════════════════════════»
                                                                            »
«     ┌───┐┌───┐┌───┐┌───┐     ┌───┐┌───┐┌───┐ ░ ┌───┐ ░ ┌───┐ ░ ┌─┐   
«q_0: ┤ Z ├┤ H ├┤ X ├┤ Z ├──■──┤ S ├┤ H ├┤ X ├─░─┤ Y ├─░─┤ H ├─░─┤M├───
«     └───┘└───┘└───┘└───┘┌─┴─┐├───┤├───┤├───┤ ░ ├───┤ ░ ├───┤ ░ └╥┘┌─┐
«q_1: ────────────────────┤ X ├┤ H ├┤ S ├┤ H ├─░─┤ I ├─░─┤ H ├─░──╫─┤M├
«                         └───┘└───┘└───┘└───┘ ░ └───┘ ░ └───┘ ░  ║ └╥┘
«c: 

/Users/yzhu/anaconda3/envs/snowflakes/lib/python3.8/site-packages/qiskit_ionq/ionq_backend.py:381: UserWarning: Option max_parallel_experiments is not used by this backend
  return super().run(circuit, **kwargs)
/Users/yzhu/anaconda3/envs/snowflakes/lib/python3.8/site-packages/qiskit_ionq/ionq_backend.py:381: UserWarning: Option memory is not used by this backend
  return super().run(circuit, **kwargs)


CB for  XY
created 2 circuits
     ┌───┐┌───┐ ░ ┌───┐     ┌───┐┌───┐┌───┐┌───┐     ┌───┐┌───┐┌───┐ ░ ┌───┐»
q_0: ┤ H ├┤ S ├─░─┤ I ├──■──┤ Z ├┤ H ├┤ X ├┤ Z ├──■──┤ S ├┤ H ├┤ X ├─░─┤ Z ├»
     ├───┤└───┘ ░ ├───┤┌─┴─┐└───┘└───┘└───┘└───┘┌─┴─┐├───┤├───┤├───┤ ░ ├───┤»
q_1: ┤ H ├──────░─┤ Y ├┤ X ├────────────────────┤ X ├┤ H ├┤ S ├┤ H ├─░─┤ Z ├»
     └───┘      ░ └───┘└───┘                    └───┘└───┘└───┘└───┘ ░ └───┘»
c: 2/═══════════════════════════════════════════════════════════════════════»
                                                                            »
«          ┌───┐┌───┐┌───┐┌───┐     ┌───┐┌───┐┌───┐ ░ ┌───┐ ░ ┌───┐┌───┐┌───┐»
«q_0: ──■──┤ Z ├┤ H ├┤ X ├┤ Z ├──■──┤ S ├┤ H ├┤ X ├─░─┤ Y ├─░─┤ S ├┤ S ├┤ S ├»
«     ┌─┴─┐└───┘└───┘└───┘└───┘┌─┴─┐├───┤├───┤├───┤ ░ ├───┤ ░ ├───┤└───┘└───┘»
«q_1: ┤ X ├────────────────────┤ X ├┤ H ├┤ S ├┤ H ├─░─┤ I ├─░─┤ I ├──────────»
«     └───┘                    └───┘└───┘└───┘└───┘ ░ └───┘ ░ └───┘          »
«c: 2/═══════════════════════

In [11]:
for i in range(len(data["XX"]["batch_0"])):
    print ((data["XX"]["batch_0"][i]))

{'n': 2, 'L': 2, 'c': 0, 'circuit': <bound method QuantumCircuit.qasm of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x14a241640>>, 'clifford': {'stabilizer': ['+XZ', '+YX'], 'destabilizer': ['+IX', '-XI']}, 'pauli': '-XX', 'repeat': 1}
{'n': 2, 'L': 2, 'c': 1, 'circuit': <bound method QuantumCircuit.qasm of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x14a266d00>>, 'clifford': {'stabilizer': ['-XZ', '+YX'], 'destabilizer': ['+IX', '+XI']}, 'pauli': 'XX', 'repeat': 1}
{'n': 2, 'L': 2, 'c': 2, 'circuit': <bound method QuantumCircuit.qasm of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x14a27d880>>, 'clifford': {'stabilizer': ['-XZ', '+YX'], 'destabilizer': ['+IX', '-XI']}, 'pauli': '-XX', 'repeat': 1}
{'n': 2, 'L': 2, 'c': 3, 'circuit': <bound method QuantumCircuit.qasm of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x14a2833a0>>, 'clifford': {'stabilizer': ['-XZ', '-YX'], 'destabilizer': ['+IX', '+XI']}, 'pauli': 'XX', 'repeat': 1}
{'n': 

## Data analysis

In [12]:
print (data.keys())
# print (data["pauli_sample_list"])
print (data["XX"].keys())
# print (type(data["XX"]["result"]))
# print (data["XX"]["result"][0].data(0))
print (data["XX"]["result"][0].get_counts())
# print (data["XX"]["result"][0].data(0)["counts"])

dict_keys(['token', 'n', 'pauli_sample_list', 'XX', 'XY', 'XZ', 'YX', 'YY', 'YZ', 'ZX', 'ZY', 'ZZ'])
dict_keys(['batch_0', 'result', 'parameters'])
[{'00': 30, '01': 52, '10': 137, '11': 1781}, {'00': 72, '01': 63, '10': 1631, '11': 234}, {'00': 135, '01': 104, '10': 1315, '11': 446}, {'00': 554, '01': 1123, '10': 148, '11': 175}, {'00': 313, '01': 311, '10': 618, '11': 758}]


In [5]:


'''Use these if read from file'''
# token = "8799418287"
# filename = "sim_cb_cnot_2022oct_" + token
# with open('/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation/EAB(code from Senrui)/data/' + filename, 'rb') as infile:
#     data = pickle.load(infile)


n = data["n"]
pauli_sample_list = data["pauli_sample_list"]
'''Specify a set of Pauli you want to estimate'''
pauli_request_list = [''.join(s) for s in itertools.product(['I','X','Y','Z'], repeat = n)] #full

fidelity_list = {} 
stdev_list = {}

for pauli_sample in pauli_sample_list:
    cb_data = data[pauli_sample]

    # n = cb_data["parameters"]['n']
    # n_total = cb_data["parameters"]['n_total'] 
    shots = cb_data["parameters"]['shots'] 
    Lrange = cb_data["parameters"]['Lrange']
    C = cb_data["parameters"]['C'] 
    eps_readout = cb_data["parameters"]['eps_readout'] 
    repeat = cb_data["parameters"]['repeat']

    cb_result = CB_process.process_CB_ionq(n, C, shots, 1, Lrange, cb_data, pauli_sample = pauli_sample,repeat=repeat, periodic=True,use_density_matrix=False, intercept_cb=False)
    raw_fidelity_list = cb_result["fidelity_list"]
    
    new_sub_label = []

    for sub_label in raw_fidelity_list.keys():
        if sub_label in fidelity_list:
            continue # wasteful!
        elif(sub_label == 'I'*n):
            fidelity_list[sub_label] = 1.0
            stdev_list[sub_label] = 0.0
        else:
            alpha, alpha_err = CB_process.fit_CB(Lrange, raw_fidelity_list[sub_label])
            fidelity_list[sub_label] = alpha
            stdev_list[sub_label] = alpha_err
            new_sub_label.append(sub_label)

    print("CB setting: ",pauli_sample[::-1]," Pauli fidelities calculated: ", [sub_label[::-1] for sub_label in new_sub_label])


# print(fidelity_list)

# print(stdev_list)
print("Parameters: n = %d, C = %d, " % (n,C), "L = ", str(Lrange))



# Average fidelity
print("Total error = ", 1-np.mean(list(fidelity_list.values())))

print("Label / Pauli infidelity / Standard deviation")
for pauli_label in pauli_request_list:
    print(pauli_label[::-1], 1-fidelity_list[pauli_label], stdev_list[pauli_label])
# print('Effective noise rate = ' + str(1-np.average(list(fidelity_list.values()))))



CB setting:  XX  Pauli fidelities calculated:  ['XI', 'IX', 'XX']
CB setting:  YX  Pauli fidelities calculated:  ['YI', 'YX']
CB setting:  ZX  Pauli fidelities calculated:  ['ZI', 'ZX']
CB setting:  XY  Pauli fidelities calculated:  ['IY', 'XY']
CB setting:  YY  Pauli fidelities calculated:  ['YY']
CB setting:  ZY  Pauli fidelities calculated:  ['ZY']
CB setting:  XZ  Pauli fidelities calculated:  ['IZ', 'XZ']
CB setting:  YZ  Pauli fidelities calculated:  ['YZ']
CB setting:  ZZ  Pauli fidelities calculated:  ['ZZ']
Parameters: n = 2, C = 1,  L =  [2, 8]
Total error =  0.0
Label / Pauli infidelity / Standard deviation
II 0.0 0.0
XI 0.0 inf
YI 0.0 inf
ZI 0.0 inf
IX 0.0 inf
XX 0.0 inf
YX 0.0 inf
ZX 0.0 inf
IY 0.0 inf
XY 0.0 inf
YY 0.0 inf
ZY 0.0 inf
IZ 0.0 inf
XZ 0.0 inf
YZ 0.0 inf
ZZ 0.0 inf


/Users/yzhu/anaconda3/envs/snowflakes/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/yzhu/anaconda3/envs/snowflakes/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/yzhu/anaconda3/envs/snowflakes/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [14]:
print (raw_fidelity_list)

{'II': {2: [1.0, 1.0, 1.0, 1.0, 1.0], 4: [1.0, 1.0, 1.0, 1.0, 1.0], 8: [1.0, 1.0, 1.0, 1.0, 1.0], 16: [1.0, 1.0, 1.0, 1.0, 1.0], 32: [1.0, 1.0, 1.0, 1.0, 1.0]}, 'IZ': {2: [0.966, -0.976, 0.961, -0.964, 0.966], 4: [0.971, 0.973, 0.977, 0.985, 0.975], 8: [0.978, 0.984, 0.98, 0.986, 0.976], 16: [0.983, 0.972, 0.98, 0.962, 0.977], 32: [0.979, 0.984, 0.984, 0.978, 0.986]}, 'ZI': {2: [-0.966, 0.976, 0.961, 0.964, 0.966], 4: [0.983, 0.978, 0.976, 0.982, 0.977], 8: [0.986, 0.981, 0.971, 0.98, 0.981], 16: [0.98, 0.989, 0.98, 0.974, 0.982], 32: [0.975, 0.979, 0.979, 0.983, 0.985]}, 'ZZ': {2: [0.966, 0.976, 0.961, 0.964, 0.966], 4: [0.954, 0.951, 0.953, 0.967, 0.952], 8: [0.964, 0.965, 0.951, 0.966, 0.957], 16: [0.963, 0.961, 0.96, 0.936, 0.959], 32: [0.956, 0.963, 0.963, 0.961, 0.971]}}


Note that, the estimation suffers from some degeneracy. E.g., $\lambda_{IZ}$ and $\lambda_{ZZ}$ cannot be individually addressed. Only their geometric mean is estimated and reported. This issue is detailed in [The learnability of Pauli noise](https://arxiv.org/abs/2206.06362).

In [6]:
error_list = fidelity_to_error(fidelity_list,n)
print("Label / Pauli error rates")
for pauli_label in pauli_request_list:
    print(pauli_label[::-1], error_list[pauli_label])

Label / Pauli error rates
II 1.0
XI 0.0
YI 0.0
ZI 0.0
IX 0.0
XX 0.0
YX 0.0
ZX 0.0
IY 0.0
XY 0.0
YY 0.0
ZY 0.0
IZ 0.0
XZ 0.0
YZ 0.0
ZZ 0.0


Similarly, in the estimation $p_{XI}=p_{XX}=0.0025$ due to degeneracy.

In [6]:
# True value for this specific noise model
eps = 0.005
def f_true(P):
    ans = 0.0
    for Pi in P:
        if Pi == 'Z' or Pi == 'Y':
            ans += 2*eps
    return ans
def p_true(P):
    if P == "XI" or P == "IX":
        return eps
    else:
        return 0.0

print("Label / True infidelity/ True Pauli error")
for P in pauli_request_list:
    print(P[::-1]," ",f_true(P)," ",p_true(P))

Label / True infidelity/ True Pauli error
II   0.0   0.0
XI   0.0   0.005
YI   0.01   0.0
ZI   0.01   0.0
IX   0.0   0.005
XX   0.0   0.0
YX   0.01   0.0
ZX   0.01   0.0
IY   0.01   0.0
XY   0.01   0.0
YY   0.02   0.0
ZY   0.02   0.0
IZ   0.01   0.0
XZ   0.01   0.0
YZ   0.02   0.0
ZZ   0.02   0.0


In [7]:
for pauli_label in pauli_request_list:
    print(pauli_label[::-1], (1-fidelity_list[pauli_label])-f_true(pauli_label))

II 0.0
XI -3.72240458179629e-05
YI 0.00011366131535684755
ZI 0.0003760411715055898
IX -8.145387763369705e-05
XX -0.00011494943944923008
YX 0.00016384049609397856
ZX 0.000473700769892069
IY 0.005677388220001753
XY 0.005370589308984939
YY -0.004615039021756844
ZY -0.0048058294033530395
IZ 0.004476540925792536
XZ 0.004580069852476396
YZ -0.0047195157624963895
ZZ -0.005029984350663801


In [1]:
print (range(5))

range(0, 5)
